Musicbrainz has a flexible API (/ws/2/recording) for fetching song data. 

Base URL: https://musicbrainz.org/ws/2/

Need to offset to paginate results (offset=25, offset=50, etc.) Might need multiple paginated calls per year to get more comprehensive data.

Watch rate limit (1 request/second).

In [ ]:
import requests
import pandas as pd

def get_musicbrainz_recordings(year, limit=25):
    base_url = "https://musicbrainz.org/ws/2/recording/"
    headers = {
        "User-Agent": "YourAppName/1.0 ( your-email@example.com )"   # can use Ruchi's or should we all have one?
    }

    params = {
        "query": f"date:[{year}-01-01 TO {year}-12-31]",
        "fmt": "json",
        "limit": limit,
        "inc": "releases+artists"
    }

    response = requests.get(base_url, headers=headers, params=params)
    data = response.json()
    recordings = data.get('recordings', [])

    songs = []
    for r in recordings:
        songs.append({
            "title": r.get("title"),
            "artist": r["artist-credit"][0]["name"] if r.get("artist-credit") else None,
            "release_date": r.get("first-release-date"),
            "id": r.get("id")
        })

    return pd.DataFrame(songs)

df_songs = get_musicbrainz_recordings(2008)
print(df_songs.head())

Lyrics from Genius API: https://genius.com/api-clients

Use Genius Search API (https://api.genius.com/search) to find songs and then scrape lyrics from song pages (since the lyrics aren't returned via API). Watch rate limits and TOS.

UPDATE-unecessary, since the Genius module Ruchi found does this autmatically...

In [65]:
import re
from bs4 import BeautifulSoup

# Genius client ID: FAUYRdGjRc5TIKn_oZLu9Pl4xBkbv8vhyqO-H83zkokyEQabXpXQkNF11vmSmVvD
# Genius client secret: lXspjKk-7i0kBR7aClzTCn-dR2_FnjALSUK5WehSP2Xe7dZMgVPVhOSRIrzagfmLDj_C8XXYqS0yOA89JdL3nA
# Genius client access token: r6vW9hyUck05RRuvBF4L1f5MjirVAeuHO0tauARd-aB1pt7xscCUVzMnFoFC2WTS
GENIUS_API_KEY = "r6vW9hyUck05RRuvBF4L1f5MjirVAeuHO0tauARd-aB1pt7xscCUVzMnFoFC2WTS"    # "your_api_key_here"


def search_genius_lyrics(song_title, artist_name):
    base_url = "https://api.genius.com/search"
    headers = {
        "Authorization": f"Bearer {GENIUS_API_KEY}"
    }
    params = {
        "q": f"{song_title} {artist_name}"
    }

    response = requests.get(base_url, headers=headers, params=params).json()
    hits = response['response']['hits']
    
    for hit in hits:
        if artist_name.lower() in hit['result']['primary_artist']['name'].lower():
            song_url = hit['result']['url']
            return song_url  # Scrape lyrics from here
    return None

def scrape_lyrics_from_url(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    lyrics_divs = soup.find_all("div", class_=re.compile("^Lyrics__Root"))
    lyrics = "\n".join([div.get_text(separator="\n") for div in lyrics_divs])
    return lyrics.strip() if lyrics else None

In [67]:
# Example usage
url = search_genius_lyrics("Blinding Lights", "The Weeknd")
lyrics = scrape_lyrics_from_url(url)
lyrics

"248 Contributors\nTranslations\nTürkçe\nSvenska\nEspañol\nSlovenčina\nРусский (Russian)\nPortuguês\nPolski\nNorsk\n日本語 (Japanese)\nItaliano\nעברית (Hebrew)\nDeutsch\nFrançais\nفارسی\nNederlands\nDansk\nShqip\nالعربية\nsrpski\nBlinding Lights Lyrics\n“Blinding Lights” is the second single to The Weeknd’s fourth studio album After Hours, following the record’s lead single “Heartless,” which released two days prior on November 27, 2019. The track finds Abel… \nRead More\n\xa0\n[Intro]\nYeah\n[Verse 1]\nI've been tryna call\nI've been on my own for long enough\nMaybe you can show me\u2005how\u2005to love, maybe\nI'm\u2005goin' through withdrawals\nYou don't even have\u2005to do too much\nYou can turn me on with just a touch, baby\n[Pre-Chorus]\nI look around and\nSin City's cold and empty (Oh)\nNo one's around to judge me (Oh)\nI can't see clearly when you're gone\n[Chorus]\nI said, ooh, I'm blinded by the lights\nNo, I can't sleep until I feel your touch\nI said, ooh, I'm drowning in the

In [ ]:
# Genius Lyrics scraping
df_songs['genius_url'] = df_songs.apply(
    lambda row: search_genius_lyrics(row['title'], row['artist']), axis=1
)
df_songs['lyrics'] = df_songs['genius_url'].apply(lambda url: scrape_lyrics_from_url(url) if url else None)

For the Discog version:
Genre and style tags are usually more accurate and consistent than MusicBrainz.

No lyrics data — still need Genius for that.

Strict rate limits: 60 requests/minute for authenticated users. Personal token: https://www.discogs.com/settings/developers

In [53]:
import requests
import pandas as pd

# Token: xmWNqVMtvKoFllImiTkEbgIMpgmdVRVggXqrWJPQ
DISCOGS_TOKEN = "xmWNqVMtvKoFllImiTkEbgIMpgmdVRVggXqrWJPQ"   #"your_discogs_token_here"

def search_discogs_by_year(year, genre=None, per_page=10):
    url = "https://api.discogs.com/database/search"
    headers = {"User-Agent": "YourAppName/1.0"}
    params = {
        "token": DISCOGS_TOKEN,
        "year": year,
        "type": "release",
        "format": "Album",
        "per_page": per_page,
        "page": 1
    }
    if genre:
        params["genre"] = genre

    response = requests.get(url, headers=headers, params=params)
    data = response.json()
    results = data.get("results", [])

    releases = []
    for r in results:                        # only desired information to put into df, BUT-
        releases.append({                    # better to get the entire JSON return and THEN extract
            "title": r.get("title"),         
            "year": r.get("year"),
            "country": r.get("country"),
            "genre": ", ".join(r.get("genre", [])),
            "style": ", ".join(r.get("style", [])),
            "artist": r.get("title").split(" - ")[0] if " - " in r.get("title", "") else None
        })

    return pd.DataFrame(releases)

df_discogs = search_discogs_by_year(2020, genre="Rock", per_page=60)
print(df_discogs.head())

                                               title  year country genre  \
0               Bible Black Tyrant - Encased in Iron  2020   Italy  Rock   
1     Jeff Scott Soto - Wide Awake (In My Dreamland)  2020   Japan  Rock   
2  Steve Perry - Traces (Alternate Versions & Ske...  2020   Japan  Rock   
3                            Metalium (2) - Tenebris  2020  Turkey  Rock   
4         Meat Crayon - Drunken doodles - Promo 2018  2020      US  Rock   

                                  style              artist  
0  Post-Metal, Doom Metal, Sludge Metal  Bible Black Tyrant  
1                             Hard Rock     Jeff Scott Soto  
2                                               Steve Perry  
3                                Thrash        Metalium (2)  
4                           Death Metal         Meat Crayon  


In [55]:
df_discogs

,title,year,country,genre,style,artist
0,Bible Black Tyrant - Encased in Iron,2020,Italy,Rock,"Post-Metal, Doom Metal, Sludge Metal",Bible Black Tyrant
1,Jeff Scott Soto - Wide Awake (In My Dreamland),2020,Japan,Rock,Hard Rock,Jeff Scott Soto
2,Steve Perry - Traces (Alternate Versions & Ske...,2020,Japan,Rock,,Steve Perry
3,Metalium (2) - Tenebris,2020,Turkey,Rock,Thrash,Metalium (2)
4,Meat Crayon - Drunken doodles - Promo 2018,2020,US,Rock,Death Metal,Meat Crayon
5,Sucia Minoria - El Imperio De Las Ratas,2020,Colombia,Rock,Punk,Sucia Minoria
6,Vedran Božić & Rock Masters - Planet Cream,2020,Croatia,Rock,"Classic Rock, Hard Rock",Vedran Božić & Rock Masters
7,Jeff Scott Soto - Wide Awake (In My Dreamland),2020,Italy,Rock,Hard Rock,Jeff Scott Soto
8,Semidigested Cum Barfing - Gurgle With Sperm A...,2020,Italy,Rock,"Goregrind, Noise",Semidigested Cum Barfing
9,Centro Da Terra - Centro Da Terra,2020,Brazil,Rock,"Psychedelic Rock, Classic Rock, Blues Rock, Ha...",Centro Da Terra


In [77]:
# After the addition of the lyrics, we should include a column that identifies song language

# Re-use Genius scraping
df_discogs['genius_url'] = df_discogs.apply(
    lambda row: search_genius_lyrics(row['title'], row['artist']), axis=1
)
df_discogs['lyrics'] = df_discogs['genius_url'].apply(lambda url: scrape_lyrics_from_url(url) if url else None)

It looks like the lyrics pull needs some more work. Need to figure out how to process the results better for analysis.

In [87]:
df_discogs.tail(10)

,title,year,country,genre,style,artist,genius_url,lyrics
50,Suppuration - The Face Rotten By Some Satanic ...,2020,Germany,Rock,Death Metal,Suppuration,None,None
51,Angel Olsen - Whole New Mess,2020,US,Rock,Indie Rock,Angel Olsen,https://genius.com/Angel-olsen-whole-new-mess-...,19 Contributors\nWhole New Mess Lyrics\n[Verse...
52,Giorgio Canali & Rossofuoco - Venti,2020,Italy,Rock,"Alternative Rock, Indie Rock",Giorgio Canali & Rossofuoco,https://genius.com/Giorgio-canali-and-rossofuo...,2 Contributors\nControvento Lyrics\n[Testo di ...
53,Décembre Noir - The Renaissance Of Hope,2020,Germany,Rock,Doom Metal,Décembre Noir,https://genius.com/Decembre-noir-hope-renaissa...,1 Contributor\nHope / Renaissance Lyrics\nAn u...
54,"Kraken Duumvirate - The Stars Below, The Seas ...",2020,Sweden,Rock,"Experimental, Doom Metal, Black Metal",Kraken Duumvirate,https://genius.com/Kraken-duumvirate-the-stars...,"1 Contributor\nThe Stars Below, The Seas Above..."
55,David Reece (2) - Cacophony Of Souls,2020,Europe,Rock,"Hard Rock, Heavy Metal",David Reece (2),None,None
56,Nubivagant - Roaring Eye,2020,Germany,Rock,Black Metal,Nubivagant,None,None
57,Proscrito - Llagas y Estigmas,2020,Spain,Rock,"Death Metal, Doom Metal",Proscrito,None,None
58,Michael Grant & The Assassins - Always The Vil...,2020,Italy,Rock,Hard Rock,Michael Grant & The Assassins,None,None
59,Shark Island - Bloodline,2020,Germany,Rock,Hard Rock,Shark Island,None,None


Found a great little module for translation. We can run the lyrics through to get laguage labels OR language probabilities (in case of K-pop type songs with multiple languages). http://pypi.org/project/translate/

In [163]:
from langdetect import detect
def safe_detect(text):
    try:
        return detect(text)
    except:
        return "unknown"

df_discogs['language'] = df_discogs['lyrics'].apply(safe_detect)

In [181]:
df_discogs

,title,year,country,genre,style,artist,genius_url,lyrics,language
0,Bible Black Tyrant - Encased in Iron,2020,Italy,Rock,"Post-Metal, Doom Metal, Sludge Metal",Bible Black Tyrant,None,None,unknown
1,Jeff Scott Soto - Wide Awake (In My Dreamland),2020,Japan,Rock,Hard Rock,Jeff Scott Soto,None,None,unknown
2,Steve Perry - Traces (Alternate Versions & Ske...,2020,Japan,Rock,,Steve Perry,None,None,unknown
3,Metalium (2) - Tenebris,2020,Turkey,Rock,Thrash,Metalium (2),None,None,unknown
4,Meat Crayon - Drunken doodles - Promo 2018,2020,US,Rock,Death Metal,Meat Crayon,None,None,unknown
5,Sucia Minoria - El Imperio De Las Ratas,2020,Colombia,Rock,Punk,Sucia Minoria,None,None,unknown
6,Vedran Božić & Rock Masters - Planet Cream,2020,Croatia,Rock,"Classic Rock, Hard Rock",Vedran Božić & Rock Masters,None,None,unknown
7,Jeff Scott Soto - Wide Awake (In My Dreamland),2020,Italy,Rock,Hard Rock,Jeff Scott Soto,None,None,unknown
8,Semidigested Cum Barfing - Gurgle With Sperm A...,2020,Italy,Rock,"Goregrind, Noise",Semidigested Cum Barfing,None,None,unknown
9,Centro Da Terra - Centro Da Terra,2020,Brazil,Rock,"Psychedelic Rock, Classic Rock, Blues Rock, Ha...",Centro Da Terra,None,None,unknown


Tips:

Use pagination (page=2, 3, ...) to gather more data per year. Might need to watch for duplicate entries.

Discogs sometimes returns artists in formats like "Beatles, The", so watch out.

Filter by format=Single for individual tracks rather than albums.